# Assignment 8

This assignement will be covered with two tutorium, you don't have to give back any work before holidays. The next part will be uploaded during the holidays and after the next tutorium after holidays, you will have to give back your work. 

## Ex. 8.0
We start by installing pytorch, a popular machine learning framework. Go to [pytorch.org](https://pytorch.org/), choose your target system and install pytorch (via conda).

You can try pytorch on [Google Colab](https://colab.research.google.com), too. Colab integrates with your Google account, your notebooks will be stored on Google Drive and you can read datasets from there too. You can even use GPUs and TPUs! However, we still suggest you install pytorch on your local computer because variables might get lost when your computer disconnects from the internet or the notebook runs idle for a while.  

Browse through the [pytorch tutorials ](https://pytorch.org/tutorials/), they often come as colab notebooks - go and execute some!

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

## Ex. 8.1
Make your own convolutionnal network of the following structure : 3 layer of convolution and 2 layer of neurones. We would like to see in it some padding, stride, pooling, drop out and batch normalization. Use the ReLU activation fonction. You should define your network, make a train fonction, a test fonction to display the result and make them work!

Setup some hyper-parameter for your network

In [ ]:
n_epochs = 3
batch_size_train = 10
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

Now, load MNIST from torch and 

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

Display some instance of the training set

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

In [ ]:
#Good luck!